In [17]:
# Импорты необходимых модулей

%matplotlib inline
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from collections import Counter
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision import transforms
from IPython import display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Загрузка обучающий и валидационного сетов данных
train_raw = pd.read_csv('./fmnist_train.csv', sep=",")
test_raw = pd.read_csv('./fmnist_test.csv', sep=',')

In [6]:
# Замена пропущенных значений на ноли
train_raw.fillna(0, inplace=True)

In [10]:
# Разделение обучающего сета на тренинговый и тестовый в нехарактерной пропорции - 0.02 к 0.98
# Это сделано для того, чтобы модель обучилась на как можно бОльшем количестве данных
# В любом случае проверка проходит на валидационном сете, а 2% данных остаются для мониторинга
# графиков loss и accuracy
msk = np.random.rand(len(train_raw)) <= 0.98
train = train_raw[msk]
test = train_raw[~msk]

In [12]:
# Форирование основых dataFrames
# Удаление стлобцов с информацией нерелевантой для обучения модели
TR = test_raw.drop(['Id'], axis=1)
X_train = train.drop(['label', 'Id'], axis=1)
y_train = train.label
X_test = test.drop(['label', 'Id'], axis=1)
y_test = test.label

Следующие функции помогают визуализацией процесса обучения сети.

In [14]:
def plot_train_process(train_loss, val_loss, train_acc, val_acc):
    '''Функция для визуализации кривых обчения (Train Loss & Train Accuracy)'''
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()

    plt.show()


# Функция обертка для обучения моделей при различных параметрах
def train_model(
    model,
    opt,
    X_train_torch,
    y_train_torch,
    X_val_torch,
    y_val_torch,
    loss_function,
    scheduler,
    n_epochs=20,
    batch_size=64,
    show_plots=True,
    eval_every=1,
    early_stopping_rounds=5
):
    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []

    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(n_epochs):
        model.train()
        local_train_loss_history = []
        local_train_acc_history = []

        permutation = torch.randperm(X_train_torch.size()[0])
        for i in range(0, X_train_torch.size()[0], batch_size):
            indices = permutation[i:i+batch_size]
            x_batch, y_batch = X_train_torch[indices], y_train_torch[indices]
            x_batch = x_batch.view(-1, 1, 28, 28)  # Преобразование в форму (N, C, H, W)

            x_batch, y_batch = x_batch.to(next(model.parameters()).device), y_batch.to(next(model.parameters()).device)

            # predict log-probabilities or logits
            y_predicted = model(x_batch)

            # compute loss
            loss = loss_function(y_predicted, y_batch)

            # compute gradients
            loss.backward()

            # Adam step
            opt.step()

            # clear gradients
            opt.zero_grad()

            local_train_loss_history.append(loss.item())
            local_train_acc_history.append(
                accuracy_score(
                    y_batch.cpu().detach().numpy(),
                    y_predicted.cpu().detach().numpy().argmax(axis=1)
                )
            )

        if epoch % eval_every == 0:
            train_loss_history.append(np.mean(local_train_loss_history))
            train_acc_history.append(np.mean(local_train_acc_history))

            model.eval()
            with torch.no_grad():
                X_val_torch_reshaped = X_val_torch.view(-1, 1, 28, 28).to(next(model.parameters()).device)
                predictions_val = model(X_val_torch_reshaped)
                val_loss = loss_function(predictions_val, y_val_torch.to(next(model.parameters()).device)).cpu().item()
                val_loss_history.append(val_loss)

                acc_score_val = accuracy_score(y_val_torch.cpu().numpy(), predictions_val.cpu().detach().numpy().argmax(axis=1))
                val_acc_history.append(acc_score_val)

                # Early stopping
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
                    if early_stopping_counter >= early_stopping_rounds:
                        print(f"Early stopping at epoch {epoch}")
                        break

                # Update scheduler
                scheduler.step(val_loss)

            if show_plots:
                display.clear_output(wait=True)
                plot_train_process(train_loss_history, val_loss_history, train_acc_history, val_acc_history)

    return model

# **Работа с моделями**

In [26]:
# Эксперементальная функция для аугментации данных
def augment_data(X_df, y_df, num_augmentations=1):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor()
    ])

    augmented_images = []
    augmented_labels = []

    for i in range(len(X_df)):
        image = X_df.iloc[i].values.reshape(28, 28).astype(np.uint8)
        label = y_df.iloc[i]

        image_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        for _ in range(num_augmentations):
            augmented_image = transform(image_tensor)
            augmented_image = augmented_image.squeeze().numpy().reshape(-1)  # Flatten the image

            augmented_images.append(augmented_image)
            augmented_labels.append(label)

    augmented_X_df = pd.DataFrame(augmented_images)
    augmented_y_df = pd.Series(augmented_labels)

    return augmented_X_df, augmented_y_df


# Аугментация данных
#augmented_X_train, augmented_y_train = augment_data(X_train, y_train)


# Объединение с исходными данными
#X_train.columns = augmented_X_train.columns
#X_train_combined = pd.concat([X_train, augmented_X_train], ignore_index=True)
#y_train_combined = pd.concat([y_train, augmented_y_train], ignore_index=True)


# Преобразование в тензоры PyTorch и инициализация устройства для обучения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)


# Определение класса обучаемой модели
class VGGStyleCNN(nn.Module):
    def __init__(self):
        super(VGGStyleCNN, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )

        self.fc_block = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=64*7*7, out_features=512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(in_features=512, out_features=10)
        )

    def forward(self, x):
        out = self.block1(x)
        out = self.block2(out)
        out = self.fc_block(out)
        return out

    def predict(self, X):
        self.eval()  # Установить модель в режим оценки
        with torch.no_grad():
            if isinstance(X, pd.DataFrame):
                X_tensor = torch.tensor(X.to_numpy(), dtype=torch.float32)
            elif isinstance(X, torch.Tensor):
                X_tensor = X
            else:
                raise ValueError("Input type not supported. Expected pandas DataFrame or torch Tensor.")

            X_tensor = X_tensor.view(-1, 1, 28, 28).to(next(self.parameters()).device)
            logits = self.forward(X_tensor)
            probabilities = F.softmax(logits, dim=1)
            predictions = torch.argmax(probabilities, dim=1)
            return predictions.cpu().numpy()

model = VGGStyleCNN().to(device)

# Определение функции потерь, оптимизатора и планировщика
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=3e-4, weight_decay=1e-6)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

In [ ]:
# Обучение модели
trained_model = train_model(
    model,
    optimizer,
    X_train_tensor,
    y_train_tensor,
    X_test_tensor,
    y_test_tensor,
    criterion,
    scheduler,
    n_epochs=35,
    batch_size=64,
    show_plots=True,
    eval_every=1,
    early_stopping_rounds=20)

In [ ]:
# Оценка accuracy на тестовом сете данных
accuracy_score(y_test, trained_model.predict(X_test))

0.9462025316455697

In [ ]:
# Получение инференса модели на валидационном сете данных
submission = trained_model.predict(TR)

In [ ]:
# Загрузка файла с ID
sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# Формирование .csv файла для загрузки на kaggle.com
sample_submission['label'] = submission
sample_submission.to_csv('first_submission.csv', index=False)
sample_submission.head(10)

,Id,label
0,0,0
1,1,1
2,2,2
3,3,2
4,4,3
5,5,6
6,6,8
7,7,6
8,8,5
9,9,0
